In [ ]:
from pathlib import Path

import geopandas as gpd
import hvplot.pandas
import numpy as np
import pandas as pd
import xarray as xr

from searvey import usgs

In [ ]:
stations_ids = [
    "02136270",
    "02171800",
    "330428079214800", 
    "021720368",
    "02172040",
]

# or from a file:
#stations = []
#with open('USGSglist_ids.txt', 'r') as f:
#    lines = f.readlines()
#    stations = [i.strip() for i in lines]
    
stations_ids

## Retrieve station metadata from ID list

In [ ]:
all_usgs_stations = usgs.get_usgs_stations()


usgs_stations = all_usgs_stations[all_usgs_stations.site_no.astype(str).isin(stations_ids)]

# See the metadata for a couple of stations
usgs_stations

In [ ]:
plot_df = usgs_stations.drop_duplicates(subset='site_no').dropna(subset=['site_no', 'dec_lat_va', 'dec_long_va'])
world_plot = plot_df.hvplot(geo=True, tiles=True, hover_cols=["site_no", "location"])
world_plot.opts(width=800, height=500)

## Retrieve USGS station data

In [ ]:
starttime = pd.to_datetime("2023-01-01T00:00:00.000-05:00")
endtime = pd.to_datetime("2023-11-10T23:59:59-05:00")
data = usgs.get_usgs_data(
    usgs_metadata=usgs_stations,
    endtime=endtime,
    period=(endtime- starttime).days,
)
data

### Filter data

In [ ]:
def drop_all_nan_coords(ds: xr.Dataset) -> xr.Dataset:
    for coord in ds.coords:
        ds = ds.isel({
            coord: np.where(
                ds.value.notnull().sum([dim for dim in ds.coords if dim != coord])
            )[0]
        })

    return ds

ds = data.sel(code='00065').sel(option='').squeeze().reset_coords()
ds = drop_all_nan_coords(ds)
ds

### Plot data

In [ ]:
ds.to_dataframe().value.hvplot(by='site_no', grid=True)

### Save to netcdf files
The dataset format is taken from a user example

In [ ]:
ft2m = 0.3048
refTime = ds.datetime.data[0]

outdir = Path('USGSdata')
outdir.mkdir(exist_ok=True, parents=True)

outfiles = []
for st in ds.site_no:
    outpath = f'{outdir}/gage{st.item()}.nc' 
    ds_st = ds.sel(site_no=st)

    ds_new = xr.Dataset(
        data_vars={
            'longitude': ('point', [ds_st.lon.data]),
            'latitude': ('point', [ds_st.lat.data]),
            'time': ('t', pd.to_timedelta(ds_st.datetime.data - refTime).total_seconds() / 3600, {'units': 'hour'}),
            'height': ('t', ds_st.value.data * ft2m, {'units': 'meter'}),
            'Data-value-qualification': ('t', np.where(ds_st.qualifier == 'A', 1, 0)),
        },
        coords={
            'point': [0],
            't': np.arange(len(ds_st.datetime))
        },
        attrs={
            'station ID': ds_st.site_no.item()
        },
    )
    
    ds_new.to_netcdf(outpath)
    outfiles.append(outpath)

### Readback the one of the netCDF files:

In [ ]:
ds_rb = xr.open_dataset(outfiles[0])
ds_rb